In [0]:
Customer_df = spark.read.format("com.crealytics.spark.excel").option("header", "true").option("inferSchema", "true").load("/mnt/dream/datalake/Dream_Super_Shop_Customer_Data.xlsx")



In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *


Correct_schema  = {
    "CustomerID": IntegerType(),
    "CustomerName":StringType(),
    "BirthDate": DateType(),
    "MaritalStatus":StringType(),
    "Gender":StringType(),
    "Occupation":StringType(),
    "Branch":StringType(),
    "PremiumCustomer":StringType()
}

#schema type check

for col_type, expected_type in Correct_schema.items():
    Customer_df = Customer_df.withColumn(f"{col_type}_is_valid", col(col_type).try_cast(expected_type).isNotNull())
    

#null check
Customer_df = Customer_df.withColumn("null_check", when(col("CustomerID").isNull() | col("CustomerName").isNull() | col("BirthDate").isNull() | col("MaritalStatus").isNull() | col("Gender").isNull() | col("Occupation").isNull() | col("Branch").isNull() | col("PremiumCustomer").isNull(),"NULL").otherwise("NOT_NULL"))

#duplicate check

window = Window.partitionBy("CustomerID")
Customer_df = Customer_df.withColumn("duplicate_check", when(count("CustomerID").over(window) > 1, "DUPLICATE").otherwise("NO_DUPLICATE"))


#taking Invalid records
invalid_records = Customer_df.filter((col("null_check") == "NULL") | (col("duplicate_check") == "DUPLICATE") | (col("CustomerID_is_valid") == False) | (col("CustomerName_is_valid") == False) | (col("BirthDate_is_valid") == False) | (col("MaritalStatus_is_valid") == False) | (col("Gender_is_valid") == False) | (col("Occupation_is_valid") == False) | (col("Branch_is_valid") == False) | (col("PremiumCustomer_is_valid") == False))


#creating table and storing
spark.sql("CREATE DATABASE IF NOT EXISTS AUDIT")

invalid_records.write.format("delta").mode("append").saveAsTable("audit.invalid_records")

valid_records = Customer_df.subtract(invalid_records)





In [0]:
%sql

select * from invalid_records;